# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
import numpy as np
import pandas as pd
from zipfile import ZipFile

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# *Unzips*

file = "Orders.zip"
with ZipFile(file, "r") as zip:
    zip.extractall("Orders")

In [3]:
orders = pd.read_csv("Orders/Orders.csv")
orders = orders.drop(columns=orders.columns[0])
orders.head()                

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
# We can get a total amount spent per customer with a groupby

customer_spent = orders.groupby(["CustomerID"]).agg({"amount_spent":sum}).reset_index()
customer_spent

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [5]:
# This way we get the groups with the quantiles specified

vip = customer_spent[customer_spent["amount_spent"] > customer_spent["amount_spent"].quantile(0.95)]
preferred = customer_spent[(customer_spent["amount_spent"] < customer_spent["amount_spent"].quantile(0.95)) & (customer_spent["amount_spent"] > customer_spent["amount_spent"].quantile(0.75))]

In [6]:
# New columns with the labels

vip["Group"] = "VIP"
vip

C:\Users\alfon\AppData\Local\Temp\ipykernel_11072\3158311743.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vip["Group"] = "VIP"


,CustomerID,amount_spent,Group
0,12346,77183.60,VIP
10,12357,6207.67,VIP
12,12359,6372.58,VIP
50,12409,11072.67,VIP
55,12415,124914.53,VIP
...,...,...,...
4207,18109,8052.97,VIP
4229,18139,8438.34,VIP
4253,18172,7561.68,VIP
4292,18223,6484.54,VIP


In [7]:
preferred["Group"] = "Preferred"
preferred

C:\Users\alfon\AppData\Local\Temp\ipykernel_11072\3077370215.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preferred["Group"] = "Preferred"


,CustomerID,amount_spent,Group
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
5,12352,2506.04,Preferred
9,12356,2811.43,Preferred
...,...,...,...
4319,18259,2338.60,Preferred
4320,18260,2643.20,Preferred
4328,18272,3078.58,Preferred
4337,18283,2094.88,Preferred


In [8]:
# We concat both DataFrames into one with all the labeled customers

groups = pd.concat([vip, preferred], axis = 0)
groups

,CustomerID,amount_spent,Group
0,12346,77183.60,VIP
10,12357,6207.67,VIP
12,12359,6372.58,VIP
50,12409,11072.67,VIP
55,12415,124914.53,VIP
...,...,...,...
4319,18259,2338.60,Preferred
4320,18260,2643.20,Preferred
4328,18272,3078.58,Preferred
4337,18283,2094.88,Preferred


In [9]:
# Then we join that DataFrame with the main one, common column CustomerID

orders = pd.merge(left = orders,
                  right = groups,
                  how = "left",
                  left_on = "CustomerID",
                  right_on = "CustomerID")

orders

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent_x,amount_spent_y,Group
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21,Preferred
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21,Preferred
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20,NaN,NaN
397920,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60,NaN,NaN
397921,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60,NaN,NaN
397922,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60,NaN,NaN


In [10]:
# And finally we define the Regular customers

orders["Group"] = orders["Group"].fillna("Regular")
orders["Group"].value_counts()

Preferred    151781
Regular      141659
VIP          104484
Name: Group, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [11]:
# We create a new table with only the VIPs

orders_vip = orders[orders["Group"] == "VIP"]
orders_vip

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent_x,amount_spent_y,Group
26,536370,22728,2010,12,3,8,alarm clock bakelike pink,24,2010-12-01 08:45:00,3.75,12583,France,90.0,7281.38,VIP
27,536370,22727,2010,12,3,8,alarm clock bakelike red,24,2010-12-01 08:45:00,3.75,12583,France,90.0,7281.38,VIP
28,536370,22726,2010,12,3,8,alarm clock bakelike green,12,2010-12-01 08:45:00,3.75,12583,France,45.0,7281.38,VIP
29,536370,21724,2010,12,3,8,panda and bunnies sticker sheet,12,2010-12-01 08:45:00,0.85,12583,France,10.2,7281.38,VIP
30,536370,21883,2010,12,3,8,stars gift tape,24,2010-12-01 08:45:00,0.65,12583,France,15.6,7281.38,VIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397883,581584,85038,2011,12,5,12,6 chocolate love heart t-lights,48,2011-12-09 12:25:00,1.85,13777,United Kingdom,88.8,25977.16,VIP
397905,581586,22061,2011,12,5,12,large cake stand hanging strawbery,8,2011-12-09 12:49:00,2.95,13113,United Kingdom,23.6,12245.96,VIP
397906,581586,23275,2011,12,5,12,set of 3 hanging owls ollie beak,24,2011-12-09 12:49:00,1.25,13113,United Kingdom,30.0,12245.96,VIP
397907,581586,21217,2011,12,5,12,red retrospot round cake tins,24,2011-12-09 12:49:00,8.95,13113,United Kingdom,214.8,12245.96,VIP


In [12]:
# Then count how many unique customers each country has

orders_vip_count = orders_vip.groupby(["Country"]).agg({"CustomerID":pd.Series.nunique}).reset_index()

In [13]:
# And select the max

orders_vip_count[orders_vip_count["CustomerID"] == orders_vip_count["CustomerID"].max()]

,Country,CustomerID
17,United Kingdom,177


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [14]:
# Following the previous approach

orders_important = orders[(orders["Group"] == "VIP") | (orders["Group"] == "Preferred")]
orders_important

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent_x,amount_spent_y,Group
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21,Preferred
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21,Preferred
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397904,581585,22466,2011,12,5,12,fairy tale cottage night light,12,2011-12-09 12:31:00,1.95,15804,United Kingdom,23.40,4206.39,Preferred
397905,581586,22061,2011,12,5,12,large cake stand hanging strawbery,8,2011-12-09 12:49:00,2.95,13113,United Kingdom,23.60,12245.96,VIP
397906,581586,23275,2011,12,5,12,set of 3 hanging owls ollie beak,24,2011-12-09 12:49:00,1.25,13113,United Kingdom,30.00,12245.96,VIP
397907,581586,21217,2011,12,5,12,red retrospot round cake tins,24,2011-12-09 12:49:00,8.95,13113,United Kingdom,214.80,12245.96,VIP


In [15]:
orders_important_count = orders_important.groupby(["Country"]).agg({"CustomerID":pd.Series.nunique}).reset_index()

In [16]:
orders_important_count[orders_important_count["CustomerID"] == orders_important_count["CustomerID"].max()]

,Country,CustomerID
26,United Kingdom,932
